In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import pandas as pd
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:

loss_fn = torch.nn.BCELoss(reduce=False, size_average=False)
input = Variable(torch.randn(1, 1))
target = Variable(torch.FloatTensor(1, 1).random_(2))
loss = loss_fn(F.sigmoid(input), target)
print(input); print(target); print(loss)

tensor([[-0.6590]])
tensor([[0.]])
tensor([[0.4170]])


In [10]:
seq_length = 5

In [15]:
class FF_NN(nn.Module):
    def __init__(self, hidden_size):
        super(Classifier, self).__init__()
        self.linear1 = nn.Linear(hidden_size, hidden_size/2)
        self.linear2 = nn.Linear(hidden_size/2, hidden_size/8)
        self.linear3 = nn.Linear(hidden_size/8, 1)
        
    def forward(self, input):
        output = self.linear1(input)
        output = self.linear2(output)
        output = self.linear3(output)
        
        return output

In [8]:
class D_RNN(nn.Module):
    def __init__(self, hidden_size):
        super(D_RNN, self).__init__()
        self.hidden_size = hidden_size
        self.gru = nn.GRU(1, hidden_size)   
            
    def forward(self, input, hidden):        
        output, hidden = self.gru(input, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [16]:
class G_RNN(nn.Module):
    def __init__(self, hidden_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.gru = nn.GRU(1, hidden_size)
        
        self.linear1 = nn.Linear(hidden_size, hidden_size/2)
        self.linear2 = nn.Linear(hidden_size/2, hidden_size/8)
        self.linear3 = nn.Linear(hidden_size/8, 1)

    def forward(self, input, hidden):  
        output, hidden = self.gru(input, hidden)
        output = self.linear1(output)
        output = self.linear2(output)
        output = self.linear3(output)
        
        return output, hidden

    def initHidden(self):
        return torch.randn(1, 1, self.hidden_size, device=device)
    
    def initInput(self):
        return torch.rand(1, 1, 1, device=device)

In [11]:
torch.full((1,), 1, device=device)

tensor([1.])

In [18]:
def train(netD, netG, clsLayer, netD_opt, netG_opt, clsLayer_opt, criterion):
    ### update discriminator ###
    # (1) train with real data # 
    
    netD.zero_grad()
    clsLayer.zero_grad()
    
    netD_hidden = netD.initHidden()
    random_tensor = torch.rand(1, 1, seq_length)
    label = torch.full((1,), 1, device=device)
    
    # pass real data through discriminator
    for i in range(seq_length):
        netD_output, netD_hidden = netD(random_tensor[i], netD_hidden)
    
    real_output = clsLayer(netD_hidden).view(-1)
    errD_real = criterion(real_output, label)
    errD_real.backward()
    
    # (2) train with fake data #
    
    netG_hidden = netG.initHidden()
    netG_input = netG.initInput()
    label.fill_(0)
    
    # generate fake data through generator
    netG_fake = torch.zeros(seg_length, 1, device=device)
    for i in range(seq_length):
        netG_input, netG_hidden = netG(netG_input, netG_hidden)
        netG_fake[i] = netG_input[0, 0]
        
    # pass fake data through discriminator
    netD_hidden = netD.initHidden()
    for i in range(seq_length):
        netD_output, netD_hidden = netD(netG_fake[i].detach(), netD_hidden)
        
    fake_output = clsLayer(netD_hidden).view(-1)
    errD_fake = criterion(fake_output, label)
    errD_fake.backward()
    
    # update discriminator graident
    netD_opt.step()
    ######
    
    
    
    ### update generator ###
    netG.zero_grad()
    label.fill_(1)
    
    # train generator to make fake data become real
    netD_hidden = netD.initHidden()
    for i in range(seq_length):
        netD_output, netD_hidden = netD(netG_fake[i], netD_hidden)
        
    output = clsLayer(netD_hidden).view(-1)
    errG = criterion(output, label)
    errG.backward()
    
    netG_opt.step()
    ######